## Evaluation of Convolutional Features for Nearest Neighbor Performance on Household Scene Recognition
For each image break into k patches. Generate convolutional features for each patch. For each patch find n nearest neighbors. Discard nearest neighbors that originate from the same image. For each patch calculate the score s which is the percent of n neighbors that have the same label. The overall score is the sum of the individual patch scored divided by the total number of pataches.

Evaluate the overall scores for different pretrained conv nets using different patch sizes and arrangments.


In [ ]:
using Metalhead
model = VGG19()
model_layers = model.layers[1:21]
y = model_layers(rand(Float32, 224, 224, 3, 1))

d = zeros(Float32, 512)

for i in 1:size(y)[1]
    for j in 1:size(y)[2]
        d += reshape(y[i,j,:], (512,))
    end
end

d /= (size(y)[1] * size(y)[2])
# des = sq_net.layers[1:35](rand(Float32, 224, 224, 3, 1))

In [8]:
using Metalhead
model = GoogleNet()
model_layers = model.layers[1:20]
y = model_layers(rand(Float64, 224, 224, 3, 1))
show(typeof(y))
size(y)

# d = zeros(Float32, 512)

# for i in 1:size(y)[1]
#     for j in 1:size(y)[2]
#         d += reshape(y[i,j,:], (512,))
#     end
# end

# d /= (size(y)[1] * size(y)[2])
# des = sq_net.layers[1:35](rand(Float32, 224, 224, 3, 1))

TrackedArray{…,Array{Float64,2}}

(1000, 1)

In [28]:
using Metalhead
model = ResNet()
model_layers = model.layers[1:20]
y = model_layers(rand(Float64, 224, 224, 3, 1))
show(typeof(y))
show(size(y))
y


TrackedArray{…,Array{Float64,2}}(2048, 1)

Tracked 2048×1 Array{Float64,2}:
 0.010937161838164436  
 0.0011053800512341482 
 0.00020313082610521173
 0.006798371811034895  
 0.0061834549981707974 
 0.009073035072006483  
 0.0                   
 2.4883882681787324e-5 
 0.00013621413610743062
 1.977902752303582e-5  
 0.002831800254030947  
 0.010630322269945475  
 0.0006660875893514395 
 ⋮                     
 1.0132694772349818e-6 
 0.006952013623774986  
 0.0024165784160882268 
 0.0                   
 4.488102484260265e-7  
 1.7331152376270306e-6 
 0.00690471385199471   
 2.494656315229674e-5  
 4.42891064563807e-6   
 0.0004814443183184906 
 2.1000655997393758e-5 
 0.0049273719837598035 

In [29]:
using Metalhead

function convert_output_(y)
    println("convert_output_ size(y) ", size(y))
    d = zeros(Float64, size(y)[3])

    for i in 1:size(y)[1]
        for j in 1:size(y)[2]
            d += reshape(y[i,j,:], (size(y)[3],))
        end
    end

    return d /= (size(y)[1] * size(y)[2])
end

function convert_output(y)
    a = Tracker.data(y)
    
    println("size(a)", size(a))
    
    y_ = permutedims(a, (4, 1, 2, 3));
    
    batch_size = size(y_)[1]
    dim_size = size(y_)[4]
    
    d = zeros(Float64, batch_size, dim_size)
    
    for i in 1:batch_size
        d[i, :] = convert_output_(y_[i, :, :, :])
    end
    
    return d[1, :]
end

model.layers[1:20]

model = ResNet()
model_layers = model.layers[1:18]
y = model_layers(rand(Float64, 224, 224, 3, 1))
println("typeof(y) ", typeof(y))
println("size(y) ", size(y))
convert_output(y)


typeof(y) TrackedArray{…,Array{Float64,4}}
size(y) (7, 7, 2048, 1)
size(a)(7, 7, 2048, 1)
convert_output_ size(y) (7, 7, 2048)


2048-element Array{Float64,1}:
 0.0006835985430226502 
 0.0                   
 0.00020890838910963546
 0.013523812669288003  
 0.0031548967268280642 
 0.018311556543845926  
 0.0                   
 4.958602991119707e-6  
 0.01742630860562222   
 0.011362998369846456  
 0.0003570370835660253 
 0.0002118991251354892 
 0.00010273428605451941
 ⋮                     
 0.00037557789863198305
 0.011317516841972698  
 1.2384030666493539e-5 
 0.00905480585357701   
 0.004040769946966463  
 5.748843893277456e-7  
 0.0                   
 0.006742436649837301  
 0.011263746751644741  
 0.011743129871428958  
 0.005326955094793563  
 0.0038862307297004004 

# VGG19

In [ ]:
using Images, Metalhead, HNSW, DelimitedFiles

function load_and_prepare_image(image_path)
    image = load(image_path);
    
    target_height = 224
    target_width = 224

    scale_percent = target_height/size(image)[1]
    scale_width = Int(floor(scale_percent*size(image)[2]))
    resized_image = imresize(image, (224, scale_width))

    margin_left = Int(fld(scale_width - target_width, 2))

    window = view(resized_image, :, margin_left+1:margin_left+target_width)
    
    return window
end

function convert_window(window)
    img = permutedims(Float32.(channelview(window)), (3, 2, 1));
    img[:, :, :] .*= 256.0;
    img[:, :, :] .-= 128.;
    img[:, :, :] ./= 128.;
end

function convert_output(y)
    d = zeros(Float32, 512)

    for i in 1:size(y)[1]
        for j in 1:size(y)[2]
            d += reshape(y[i,j,:], (512,))
        end
    end

    return d /= (size(y)[1] * size(y)[2])    
end

function prep_model_layers()
    model = VGG19()
    return model.layers[1:21]
end

function prepare_image_paths_labels()
    labels_file = "household_images/labels.csv"
    data = DelimitedFiles.readdlm(labels_file, ',', header=true)[1]

    directory_path = "household_images/images"
    image_paths = [joinpath(directory_path, data[y, 1]) for y in 1:size(data)[1]]
    image_labels = [data[y, 2] for y in 1:size(data)[1]]
    
    return image_paths, image_labels
end

function main()
   
    model_layers = prep_model_layers()
    
    image_paths, image_labels = prepare_image_paths_labels()

    image_count = size(image_paths)[1]
    window_count = 1
    neighbor_count = 3
    
    descriptors = []
    labels = []
    indexes = []

    scores = []
    
    # build a dict of descritor index to image index
    
    for i in 1:image_count
        
        image_path = image_paths[i]
        
        println(i, " ", image_path)
        
        window = load_and_prepare_image(image_path)
        
        batch = zeros(Float32, 224, 224, 3, window_count)
        
        batch[:,:,:,1] = convert_window(window)

        #batch[:,:,:,1] = rand(Float32, 224, 224, 3)
        
        y = model_layers(batch)
        des = convert_output(y)
        
        push!(descriptors, des)
        push!(labels, image_labels[i])
        push!(indexes, i)
    
    end
    
    hnsw = HierarchicalNSW(descriptors; efConstruction=100, M=16, ef=50)
    add_to_graph!(hnsw)
    
    # for each descriptor find neighbors, enouph to usually include all candidates
    for i in 1:size(descriptors)[1]
        idxs, dists = knn_search(hnsw, descriptors[i], neighbor_count + window_count)
        
        # find neighbor_count number of candidates and count how many of those belong to the same label
        
        count = 0
        candidates = 0
        
        for j in 1:size(idxs)[1]
            idx = idxs[j]
            
            # if neighbor belongs to same picture, it is not a candidate
            if indexes[i] == indexes[idx]
                continue
            end
            
            # this is a candidate
            
            candidates += 1
            
            # if neighbor has same label
            
            if labels[i] == labels[idx]
                count += 1
            end
            
            if candidates >= neighbor_count
                break
            end
        end
        
        println("Score: ", count/candidates)
        
        push!(scores, count/candidates)
        
    end
    
    return sum(scores)/size(scores)[1], scores
end

main()

# ResNet50

In [2]:
using Images, Metalhead, HNSW, DelimitedFiles, Flux.Tracker

function load_and_prepare_image(image_path)
    image = load(image_path);
    
    target_height = 224
    target_width = 224

    scale_percent = target_height/size(image)[1]
    scale_width = Int(floor(scale_percent*size(image)[2]))
    resized_image = imresize(image, (224, scale_width))

    margin_left = Int(fld(scale_width - target_width, 2))

    window = view(resized_image, :, margin_left+1:margin_left+target_width)
    
    return window
end

function convert_window(window)
    img = permutedims(Float64.(channelview(window)), (3, 2, 1));
    img[:, :, :] .*= 256.0;
    img[:, :, :] .-= 128.;
    img[:, :, :] ./= 128.;
end

function convert_output(y)
    a = Tracker.data(y)
    b = reshape(a, (size(y)[1],))
    return b
end

# function convert_output_(y)
#     println("convert_output_ size(y) ", size(y))
#     d = zeros(Float64, size(y)[3])

#     for i in 1:size(y)[1]
#         for j in 1:size(y)[2]
#             d += reshape(y[i,j,:], (size(y)[3],))
#         end
#     end

#     return d /= (size(y)[1] * size(y)[2])
# end

# function convert_output(y)
#     a = Tracker.data(y)
    
#     println("size(a)", size(a))
    
#     y_ = permutedims(a, (4, 1, 2, 3));
    
#     batch_size = size(y_)[1]
#     dim_size = size(y_)[4]
    
#     d = zeros(Float64, batch_size, dim_size)
    
#     for i in 1:batch_size
#         d[i, :] = convert_output_(y_[i, :, :, :])
#     end
    
#     return d[1, :]
# end

function prep_model_layers()
    model = ResNet()
    return model.layers[1:20]
end

function prepare_image_paths_labels()
    labels_file = "household_images/labels.csv"
    data = DelimitedFiles.readdlm(labels_file, ',', header=true)[1]

    directory_path = "household_images/images"
    image_paths = [joinpath(directory_path, data[y, 1]) for y in 1:size(data)[1]]
    image_labels = [data[y, 2] for y in 1:size(data)[1]]
    
    return image_paths, image_labels
end

function main()
   
    model_layers = prep_model_layers()
    
    image_paths, image_labels = prepare_image_paths_labels()

    image_count = 58 #size(image_paths)[1]
    window_count = 1
    neighbor_count = 3
    
    descriptors = []
    labels = []
    indexes = []

    scores = []
    
    # build a dict of descritor index to image index
    
    for i in 1:image_count
        
        image_path = image_paths[i]
        
        println(i, " ", image_path)
        
        window = load_and_prepare_image(image_path)
        
        batch = zeros(Float64, 224, 224, 3, window_count)
        
        batch[:,:,:,1] = convert_window(window)

        #batch[:,:,:,1] = rand(Float64, 224, 224, 3)
        
        y = model_layers(batch)
        des = convert_output(y)
        
        push!(descriptors, des)
        push!(labels, image_labels[i])
        push!(indexes, i)
    
    end
    
    hnsw = HierarchicalNSW(descriptors; efConstruction=100, M=16, ef=50)
    add_to_graph!(hnsw)
    
    # for each descriptor find neighbors, enouph to usually include all candidates
    for i in 1:size(descriptors)[1]
        idxs, dists = knn_search(hnsw, descriptors[i], neighbor_count + window_count)
        
        # find neighbor_count number of candidates and count how many of those belong to the same label
        
        count = 0
        candidates = 0
        
        for j in 1:size(idxs)[1]
            idx = idxs[j]
            
            # if neighbor belongs to same picture, it is not a candidate
            if indexes[i] == indexes[idx]
                continue
            end
            
            # this is a candidate
            
            candidates += 1
            
            # if neighbor has same label
            
            if labels[i] == labels[idx]
                count += 1
            end
            
            if candidates >= neighbor_count
                break
            end
        end
        
        println("Score: ", count/candidates)
        
        push!(scores, count/candidates)
        
    end
    
    return sum(scores)/size(scores)[1], scores
end

main()

1 household_images/images/IMG_0317.jpg
2 household_images/images/IMG_0332.jpg
3 household_images/images/IMG_0333.jpg
4 household_images/images/IMG_0334.jpg
5 household_images/images/IMG_0452.jpg
6 household_images/images/IMG_0453.jpg
7 household_images/images/IMG_0454.jpg
8 household_images/images/IMG_0456.jpg
9 household_images/images/IMG_0461.jpg
10 household_images/images/IMG_0464.jpg
11 household_images/images/IMG_0465.jpg
12 household_images/images/IMG_0470.jpg
13 household_images/images/IMG_0559.jpg
14 household_images/images/IMG_0560.jpg
15 household_images/images/IMG_0585.jpg
16 household_images/images/IMG_0586.jpg
17 household_images/images/IMG_0682.jpg
18 household_images/images/IMG_0683.jpg
19 household_images/images/IMG_0685.jpg
20 household_images/images/IMG_0688.jpg
21 household_images/images/IMG_0746.jpg
22 household_images/images/IMG_0747.jpg
23 household_images/images/IMG_1035.jpg
24 household_images/images/IMG_1048.jpg
25 household_images/images/IMG_1052.jpg
26 househ

(0.4195402298850573, Any[0.333333, 0.333333, 0.333333, 0.666667, 0.333333, 0.333333, 0.666667, 0.666667, 0.333333, 0.0  …  0.666667, 0.333333, 0.666667, 1.0, 0.333333, 0.0, 0.0, 0.0, 0.333333, 0.0])

## Scores for Random Data
0.04370860927152313
0.03487858719646797
0.04591611479028694
0.034437086092715216
0.05209713024282561

## Scores for Image Data

### VGG19 - Layers 1:21

#### Whole Image (1 Patch)

3 Nearest Neighbors
0.5863134657836648


### Squeeze Net - Layers 1:35

#### Whole Image (1 Patch)

3 Nearest Neighbors
0.1580573951434876

7 Nearest Neighbors
0.12128666035950819


### ResNet50
0.31609195402298834

#### Whole Image (1 Patch)

3 Nearest Neighbors
0.16997792494481195


### GoogleNet

#### Whole Image (1 Patch)

3 Nearest Neighbors
0.21986754966887403

#### Whole Image (12 Patch)

3 Nearest Neighbors
0.16401766004414994

# GoogleNet

In [2]:
using Images, Metalhead, DelimitedFiles, Flux.Tracker, HNSW #NearestNeighbors 

function load_and_prepare_image(image_path)
    image = load(image_path);
    
    tile_size = 224
    vertical_tiles = 2
    
    target_height = tile_size * vertical_tiles
    scale_percent = target_height/size(image)[1]
    
    horizontal_tiles = Int(fld(size(image)[2] * scale_percent, tile_size))
    target_width = tile_size * horizontal_tiles
    
    scale_width = Int(floor(scale_percent*size(image)[2]))
    resized_image = imresize(image, (target_height, scale_width))

    margin_left = Int(fld(scale_width - target_width, 2))

    windows = []
    
    for i in 1:vertical_tiles
        for j in 1:horizontal_tiles
            window = view(resized_image, ((i-1)*tile_size+1):(i*tile_size), (margin_left+(j-1)*tile_size+1):(margin_left+j*tile_size))
            #println(size(window))
            push!(windows, window)
        end
    end

    return windows
end
    
function convert_window(window)
    img = permutedims(Float64.(channelview(window)), (3, 2, 1));
    img[:, :, :] .*= 256.0;
    img[:, :, :] .-= 128.;
    img[:, :, :] ./= 128.;
end

function convert_output(y)
    a = Tracker.data(y)
    b = permutedims(a, (2, 1));
    return b
end

function prep_model_layers()
    model = GoogleNet()
    return model.layers[1:18]
end

function prepare_image_paths_labels()
    labels_file = "household_images/labels.csv"
    data = DelimitedFiles.readdlm(labels_file, ',', header=true)[1]

    directory_path = "household_images/images"
    image_paths = [joinpath(directory_path, data[y, 1]) for y in 1:size(data)[1]]
    image_labels = [data[y, 2] for y in 1:size(data)[1]]
    
    return image_paths, image_labels
end

function main()
   
    model_layers = prep_model_layers()
    
    image_paths, image_labels = prepare_image_paths_labels()

    image_count = 3 # size(image_paths)[1]
    neighbor_count = 3
    
    descriptors = []
    labels = []
    indexes = []

    scores = []
    
    # build a dict of descritor index to image index
    
    window_count = 0
    
    for i in 1:image_count
        
        image_path = image_paths[i]
        
        println(i, " ", image_path)
        
        windows = load_and_prepare_image(image_path)
        
        window_count = size(windows)[1]
        
        batch = zeros(Float64, 224, 224, 3, window_count)
        
        for j in 1:window_count
            batch[:,:,:,j] = convert_window(windows[j])
        end
        
        #batch[:,:,:,1] = rand(Float64, 224, 224, 3)

        y = model_layers(batch)
        des = convert_output(y)

        for j in 1:window_count
            push!(descriptors, des[j, :])
            push!(labels, image_labels[i])
            push!(indexes, i)
        end
    
    end
    
    hnsw = HierarchicalNSW(descriptors; efConstruction=100, M=16, ef=50)
    add_to_graph!(hnsw)
    
#     _descriptors = hcat(descriptors...)
#     println("size(_descriptors)", size(_descriptors))
#     balltree = BallTree(_descriptors, Euclidean())
    
    # for each descriptor find neighbors, enouph to usually include all candidates
    for i in 1:size(descriptors)[1]
        idxs, dists = knn_search(hnsw, descriptors[i], neighbor_count + window_count)
        
        println(dists)
        
#         println("size(descriptors[i])", size(descriptors[i]))
#         idxs, dists = knn(balltree, descriptors[i], neighbor_count + window_count, sortres = true) 
        
        # find neighbor_count number of candidates and count how many of those belong to the same label
        
        count = 0
        candidates = 0
        
        for j in 1:size(idxs)[1]
            idx = idxs[j]
            
            # if neighbor belongs to same picture, it is not a candidate
            if indexes[i] == indexes[idx]
                continue
            end
            
            # this is a candidate
            
            candidates += 1
            
            # if neighbor has same label
            
            if labels[i] == labels[idx]
                count += 1
            end
            
            if candidates >= neighbor_count
                break
            end
        end
        
        println("Score: ", count/candidates)
        
        push!(scores, count/candidates)
        
    end
    
    return sum(scores)/size(scores)[1], scores
end

main()

1 household_images/images/IMG_0317.jpg
2 household_images/images/IMG_0332.jpg
3 household_images/images/IMG_0333.jpg
[0.0, 4.55424, 6.29004, 6.48131, 6.92869, 7.0008, 7.50264]
Score: 1.0
[0.0, 3.56944, 4.07132, 4.21839, 4.55424, 5.75837, 5.94116]
Score: 1.0
[0.0, 7.95873, 9.06713, 9.16972, 9.50789, 9.99824, 10.7554]
Score: 1.0
[0.0, 7.93917, 8.55173, 8.77406, 8.99288, 9.18322, 9.33598]
Score: 1.0
[0.0, 5.51671, 7.0008, 8.56876, 9.16972, 9.33598, 10.3882]
Score: 1.0
[0.0, 1.95461, 3.18063, 4.07132, 5.01399, 5.46564, 6.48131]
Score: 1.0
[0.0, 1.03924, 4.5085, 4.98014, 5.01399, 5.75837, 8.48401]
Score: 1.0
[0.0, 2.77131, 3.18063, 4.21839, 4.5085, 4.77239, 6.92869]
Score: 1.0
[0.0, 10.0408, 10.9216, 10.9605, 11.1365, 11.4087, 11.8844]
Score: 1.0
[0.0, 5.51671, 7.50264, 9.38854, 9.8046, 9.99824, 11.0176]
Score: 1.0
[0.0, 1.95461, 2.77131, 3.56944, 4.98014, 5.37428, 6.29004]
Score: 1.0
[0.0, 1.03924, 4.77239, 5.37428, 5.46564, 5.94116, 8.51059]
Score: 1.0


(1.0, Any[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])

# VGG19 v2

In [ ]:
using Images, Metalhead, HNSW, DelimitedFiles, Flux.Tracker

function load_and_prepare_image(image_path)
    image = load(image_path);
    
    tile_size = 224
    vertical_tiles = 2
    
    target_height = tile_size * vertical_tiles
    scale_percent = target_height/size(image)[1]
    
    horizontal_tiles = Int(fld(size(image)[2] * scale_percent, tile_size))
    target_width = tile_size * horizontal_tiles
    
    scale_width = Int(floor(scale_percent*size(image)[2]))
    resized_image = imresize(image, (target_height, scale_width))

    margin_left = Int(fld(scale_width - target_width, 2))

    windows = []
    
    for i in 1:vertical_tiles
        for j in 1:horizontal_tiles
            window = view(resized_image, ((i-1)*tile_size+1):(i*tile_size), (margin_left+(j-1)*tile_size+1):(margin_left+j*tile_size))
            #println(size(window))
            push!(windows, window)
        end
    end

    return windows
end
    
function convert_window(window)
    img = permutedims(Float32.(channelview(window)), (3, 2, 1));
    img[:, :, :] .*= 256.0;
    img[:, :, :] .-= 128.;
    img[:, :, :] ./= 128.;
end

function convert_output_(y)
    d = zeros(Float32, 512)

    for i in 1:size(y)[1]
        for j in 1:size(y)[2]
            d += reshape(y[i,j,:], (512,))
        end
    end

    return d /= (size(y)[1] * size(y)[2])
end

function convert_output(y)
    y_ = permutedims(y, (4, 1, 2, 3));
    
    batch_size = size(y_)[1]
    
    d = zeros(Float32, batch_size, 512)
    
    for i in 1:batch_size
        d[i, :] = convert_output_(y_[i, :, :, :])
    end
    
    return d
end

function prep_model_layers()
    model = VGG19()
    return model.layers[1:21]
end

function prepare_image_paths_labels()
    labels_file = "household_images/labels.csv"
    data = DelimitedFiles.readdlm(labels_file, ',', header=true)[1]

    directory_path = "household_images/images"
    image_paths = [joinpath(directory_path, data[y, 1]) for y in 1:size(data)[1]]
    image_labels = [data[y, 2] for y in 1:size(data)[1]]
    
    return image_paths, image_labels
end

function main()
   
    model_layers = prep_model_layers()
    
    image_paths, image_labels = prepare_image_paths_labels()

    image_count = size(image_paths)[1]
    neighbor_count = 3
    
    descriptors = []
    labels = []
    indexes = []

    scores = []
    
    # build a dict of descritor index to image index
    
    window_count = 0
    
    for i in 1:image_count
        
        image_path = image_paths[i]
        
        println(i, " ", image_path)
        
        windows = load_and_prepare_image(image_path)
        
        window_count = size(windows)[1]
        
        batch = zeros(Float32, 224, 224, 3, window_count)
        
        for j in 1:window_count
            batch[:,:,:,j] = convert_window(windows[j])
        end
        
        #batch[:,:,:,1] = rand(Float64, 224, 224, 3)

        y = model_layers(batch)
        des = convert_output(y)

        for j in 1:window_count
            push!(descriptors, des[j, :])
            push!(labels, image_labels[i])
            push!(indexes, i)
        end
    
    end
    
    
    hnsw = HierarchicalNSW(descriptors; efConstruction=100, M=16, ef=50)
    add_to_graph!(hnsw)
    
    # for each descriptor find neighbors, enouph to usually include all candidates
    for i in 1:size(descriptors)[1]
        
        println(shape(descriptors[i]))
        
        idxs, dists = knn_search(hnsw, descriptors[i], neighbor_count + window_count)
        
        # find neighbor_count number of candidates and count how many of those belong to the same label
        
        count = 0
        candidates = 0
        
        for j in 1:size(idxs)[1]
            idx = idxs[j]
            
            # if neighbor belongs to same picture, it is not a candidate
            if indexes[i] == indexes[idx]
                continue
            end
            
            # this is a candidate
            
            candidates += 1
            
            # if neighbor has same label
            
            if labels[i] == labels[idx]
                count += 1
            end
            
            if candidates >= neighbor_count
                break
            end
        end
        
        println("Score: ", count/candidates)
        
        push!(scores, count/candidates)
        
    end
    
    return sum(scores)/size(scores)[1], scores
end

main()

In [ ]:
using Pkg
Pkg.add("NearestNeighbors")

In [ ]:
using NearestNeighbors

balltree = BallTree([0. 0. ; 1. 1.], Euclidean())
idxs, dists = knn(balltree, reshape([0. 0.], (2, 1)), 2, true) 

In [ ]:
[0 0;1 1]

In [ ]:
reshape([0. 0.], (2, 1))

In [ ]:
?phi

In [4]:
using Pkg
Pkg.add("FLANN")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...


Pkg.Types.ResolverError: Unsatisfiable requirements detected for package FLANN [4ef67f76]:
 FLANN [4ef67f76] log:
 ├─possible versions are: 0.0.1-0.0.4 or uninstalled
 ├─restricted to versions * by an explicit requirement, leaving only versions 0.0.1-0.0.4
 └─restricted by julia compatibility requirements to versions: uninstalled — no versions left